In [1]:
import nltk as nl
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import WordNetLemmatizer
import pickle
import pandas as pd
from langdetect import detect
from collections import defaultdict
import numpy as np
import heapq
import re

In [2]:
with open( 'vocabulary.pkl', 'rb') as f:
        vocabulary= pickle.load(f)
        f.close()

In [3]:
with open( 'Inverted_Index.pkl', 'rb') as f:
        ID= pickle.load(f)
        f.close()

In [4]:
df=pd.read_csv('Dataset_final_filtered.tsv', sep='\t')

In [5]:
df

,document_ID,bookTitle,bookSeries,bookAuthors,ratingValue,ratingCount,reviewCount,Plot_Values,Plot,NumberofPages,Publishing_Date,Characters,Setting,Url
0,0,MARS,NaN,Jasmine Rose,4.38,69,13,"{5: '47', 4: '7', 3: '11', 2: '2', 1: '2'}",❝�� my heart has become a planetand you are th...,NaN,2014,NaN,NaN,https://www.goodreads.com/book/show/23279048-mars
1,1,Black Box,NaN,Cassia Leo,4.02,6244,903,"{5: '2297', 4: '2320', 3: '1181', 2: '345', 1:...",♥️ Three fateful encounters....♥️ Two heart-br...,400.0,February 28th 2014,"['Mikki Gladstone', 'William ""Crush"" Slayer']","['Boston, Massachusetts']",https://www.goodreads.com/book/show/29539518-b...
2,2,Ruin and Rising,The Shadow and Bone Trilogy #3,Leigh Bardugo,4.09,158624,19396,"{5: '62107', 4: '59607', 3: '27962', 2: '6810'...",▶ \nAlternative Cover Edition #1\nThe capital ...,422.0,June 17th 2014,"['Alina Starkov', 'Malyen Oretsev', 'Darkling'...",['Ravka '],https://www.goodreads.com/book/show/14061957-r...
3,3,Shadow and Bone,The Shadow and Bone Trilogy #1,Leigh Bardugo,3.99,313190,32760,"{5: '108466', 4: '121286', 3: '62748', 2: '146...",▶ \nAlternative Cover Edition #1\nSurrounded b...,358.0,June 5th 2012,"['Alina Starkov', 'Malyen Oretsev', 'Darkling'...",['Ravka '],https://www.goodreads.com/book/show/10194157-s...
4,4,An Ember in the Ashes,An Ember in the Ashes #1,Sabaa Tahir,4.27,184959,23147,"{5: '91602', 4: '63735', 3: '21164', 2: '5487'...",▶ \nAlternative Cover Edition #1\nLaia is a sl...,446.0,February 9th 2016,"['Laia', 'Elias Veturius', 'Helene Aquilla', '...",NaN,https://www.goodreads.com/book/show/27774758-a...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26237,26237,Breathing Room,NaN,Marsha Hayles,4.19,1797,326,"{5: '794', 4: '632', 3: '306', 2: '51', 1: '14'}",\n\n\nEvvy Hoffmeister is thirteen years old w...,256.0,June 5th 2012,NaN,['Minnesota'],https://www.goodreads.com/book/show/13037526-b...
26238,26238,Eve of Destruction,Marked #2,S.J. Day,3.97,2764,135,"{5: '914', 4: '1066', 3: '607', 2: '132', 1: '...","\n\n\nClass is in, but Evangeline Hollis is st...",414.0,June 2nd 2009,NaN,['United States of America'],https://www.goodreads.com/book/show/2332154.Ev...
26239,26239,Time and Chance,Henry II and Eleanor of Aquitaine #2,Sharon Kay Penman,4.29,7829,356,"{5: '3699', 4: '2899', 3: '1085', 2: '115', 1:...","\n\n\n\nIn When Christ and His Saints Slept, a...",544.0,February 4th 2003,"['Thomas Becket', 'Eleanor of Aquitaine', 'Hen...",['England'],https://www.goodreads.com/book/show/234560.Tim...
26240,26240,An Autumn War,Long Price Quartet #3,Daniel Abraham,4.03,6331,248,"{5: '2143', 4: '2610', 3: '1264', 2: '224', 1:...",\n\n\n\nDaniel Abraham delighted fantasy reade...,366.0,July 22nd 2008,NaN,NaN,https://www.goodreads.com/book/show/2443516.An...


In [6]:
def clean_info (string):
    # this command will split the string given in input in substrings by using 
    # the words given to RegexpTokenizer as argument
    

    
    # filter the punctuation
    tmp = nl.RegexpTokenizer(r"['\w-]+").tokenize(string)  
    
    #first we lower all the words otherwise words such as AND,IS,MY are not consider stopwords 
    tmp = [word.lower() for word in tmp]
    
     # filter the stopwords (e.g. 'the', 'my', 'an', ...)
    tmp = [word for word in tmp if not word in stopwords.words("english")]
    
    #we lemmatize all the words (e.g. 'dirn')
    lemma = WordNetLemmatizer()
    tmp = [lemma.lemmatize(word, pos = "v") for word in tmp]    # v for verbs
    tmp = [lemma.lemmatize(word, pos = "n") for word in tmp]    # n for nouns
    
    
    final = ' '.join(tmp)
    
    return final

In [7]:

# we will need first these two functions to implement the search function 


# this function takes as input a list of lists and gives back the index of the list that has minimus first element

def find_min_list (L):
    min_elem = L[0][0]
    count = 0
    index = 0
    for l in L[1:]: 
        if min_elem > l[0]:
            count = count + 1 
            index = count 
            min_elem = l[0]
        else: 
            count = count + 1 

    return index 
        

# this function takes as input a list of lists and gives back the list created from the intersection of the lists 
    
def intersect_list (L):
    results = []
    while all(len(l) > 0 for l in L):

        if all([L[0][0] == l[0] for l in L[1:]]):     
            results.append(L[0][0])
            L = [l[1:] for l in L]

        else : 
            min_index = find_min_list(L)
            L[min_index] = L[min_index][1:]

    return results

In [8]:
def find_query(inverted_index, vocabulary):
    
    string = input() # asks the user a string of words to look up 
    
    string_cleaned = clean_info(string)
    list_words = string_cleaned.split(' ')
    
    
    # now translate the list of words in term_id )
    list_termID = []
    for word in list_words: 
        list_termID.append(vocabulary.get(word))    
        
   

    # retrieve the documents in the inverted index and collect them in a list 
    list_documents = []
    for term_id in list_termID:
        if term_id in inverted_index.keys():   # checking if the word we are looking for is in the inverted_index
            list_documents.append(inverted_index.get(term_id))
    
     
    # now intersect these lists (HERE IT'S FUNDAMENTAL TO SUPPOSE THAT THE DOCUMENTS ARE COLLECTED AS INCREASING 
    # SEQUENCES)
    results = intersect_list(list_documents)
    
    
    return results

In [9]:
results = find_query(ID, vocabulary)

fire magic


In [10]:
# to print the plots with the newlines
pd.set_option('display.max_colwidth', None)

In [11]:
# we still have to print the table without the newlines 
df.iloc[results][['bookTitle', 'Plot', 'Url']]

,bookTitle,Plot,Url
117,Druids,"“Mine was the vast dark sky and the spaces between the stars that called out to me; mine was the promise of magic.”So spoke the young Celt Ainvar, centuries before the enchanted age of Arthur and Merlin. An orphan taken in by the chief druid of the Carnutes in Gaul, Ainvar possessed talents that would lead him to master the druid mysteries of thought, healing, magic, and battle— talents that would make him a soul friend to the Prince Vercingetorix . . . though the two youths were as different as fire and ice.Yet Ainvar’s destiny lay with Vercingetorix, the sun-bright warrior-king. Together they traveled through bitter winters and starlit summers in Gaul, rallying the splintered Celtic tribes against the encroaching might of Julius Caesar and the soulless legions of Rome. . . .From the Paperback edition.",https://www.goodreads.com/book/show/88009.Druids
175,Tales of a Female Nomad: Living at Large in the World,"“I move throughout the world without a plan, guided by instinct, connecting through trust, and constantly watching for serendipitous opportunities.” —From the PrefaceTales of a Female Nomad is the story of Rita Golden Gelman, an ordinary woman who is living an extraordinary existence. At the age of forty-eight, on the verge of a divorce, Rita left an elegant life in L.A. to follow her dream of connecting with people in cultures all over the world. In 1986 she sold her possessions and became a nomad, living in a Zapotec village in Mexico, sleeping with sea lions on the Galapagos Islands, and residing everywhere from thatched huts to regal palaces. She has observed orangutans in the rain forest of Borneo, visited trance healers and dens of black magic, and cooked with women on fires all over the world. Rita’s example encourages us all to dust off our dreams and rediscover the joy, the exuberance, and the hidden spirit that so many of us bury when we become adults.",https://www.goodreads.com/book/show/85622.Tales_of_a_Female_Nomad
1088,The Fire,"Whit and Wisty Allgood have sacrificed everything to lead the resistance against the merciless totalitarian regime that governs their world. Its supreme leader, The One Who Is The One, has banned everything they hold dear: books, music, art, and imagination. But the growing strength of the siblings' magic hasn't been enough to stop the One's evil rampage, and now he's executed the only family they had left.Wisty knows that the time has finally come for her to face The One. But her fight and her fire only channel more power to this already invincible being. How can she and Whit possibly prepare for their imminent showdown with the ruthless villain that devastated their world-before he can truly become all-powerful?In this stunning climax of the epic Witch &amp; Wizard series, the stakes have never been higher-and the consequences will change everything.",https://www.goodreads.com/book/show/10778410-the-fire
1595,Dragon Prince,"When Rohan became the new prince of the Desert, ruler of the kingdom granted to his family for as long as the Long Sands spewed fire, he took the crown with two goals in mind. First and foremost, he sought to bring permanent peace to his world of divided princedoms. And, in a land where dragon-slaying was a proof of manhood, Rohan was the sole champion of the dragons, fighting desperately to preserve the last remaining lords of the sky and with them a secret which might be the salvation of his people.... Sioned, the Sunrunner witch who was fated by Fire to be Rohan’s bride, had mastered the magic of sunlight and moonglow, catching hints of a yet to be formed pattern which could irrevocably affect the destinies of Sunrunners and ordinary mortals alike. Yet caught in the machinations of the Lady of Goddess Keep, and of Prince Rohan and his sworn enemy, the treacherously cunning High Prince, could Sioned alter this crucial pattern to protect her lord from the menace of a war that threatened to set the land ablaze?",https://www.goodr

In [12]:
#save the results in a dataset
dataset_query=df.iloc[results]

In [13]:
def find_authors (df):
    authors = []
    
    for x in df.bookAuthors:
        
        if '[' in x:                # case with multiple authors
            list_authors = x.split(',')
    
            
            for author in list_authors[:-1]:
                authors.append(author[2:-1])
            authors.append(list_authors[-1][2:-2])
        
        else: 
            authors.append(x)
    
    return authors
def find_authors_popularity (list_authors_small, list_authors_big):
    
    dict_authors = {x:list_authors_big.count(x) for x in list_authors_small}
    
    return dict_authors
def year(y):
    try:
        y.split(' ')[-1]
        return y
    except:
        return 0
def new_score_func(results_query):
    
    #initialize the new_score
    new_score=[]
    for d in results_query:
        new_score.append([0,d])
    
    #rows that we need from the dataset
    dataset_query=df.iloc[results_query]
    n=len(dataset_query)
    
    titles=dataset_query.apply(lambda x : x['bookTitle'].lower(),axis=1)
    
    pages=dataset_query['NumberofPages']
    
    autors=dataset_query['bookAuthors']
    
    years_pub=dataset_query.apply(lambda x : year(x),axis=1)
    
    
    # 1. score based on title 
    query_title=str.lower(input('1. Qual è il titolo del tuo libro? Inserisci le parole chiave del titolo che stai cercando,altrimenti premi invio (e.g. Harry)\n').strip(".,;'"))
    
    #we add a score according to the number of words of the title found
    if query_title!='':
        query_title=query_title.split(' ')
        n_query=len(query_title)
        for i in range(n):
            count=0
            for q in query_title:
                if q in titles.iloc[i]:
                    count+=1
            new_score[i][0]+=count/n_query
        
    # 2. score based on number of pages
    
    range_pagine=input('2. Quante pagine ha il libro che cerchi?\
Inserisci un range di pagine,altrimenti premi invio (e.g. 200-400)\n')
    
    if range_pagine!='':
        minimo_p,massimo_p=range_pagine.split('-')
        
        # if the user exchange minimum and maximum
        if minimo_p>massimo_p:
            minimo_p,massimo_p=massimo_p,minimo_p
            
        # if the number of pages falls in the range we add a score
        for i in range(n):
            if int(minimo_p)<pages.iloc[i]<int(massimo_p):
                new_score[i][0]+=0.5
        
    # 3. score based on year of publication 
    
    range_years=input('3. Quando è stato pubblicato il libro?\
Inserisci un range di anni,altrimenti premi invio(e.g. 1990-2010)\n')
    
    if range_years!='':
        minimo_y,massimo_y=range_years.split('-')
        
        # if the user exchange minimum and maximum
        if minimo_y>massimo_y:
            minimo_y,massimo_y=massimo_y,minimo_y
        
        # if the publication year falls in the range we add a score
        for i in range(n):
            if int(minimo_y)<int(years_pub.iloc[i])<int(massimo_y):
                new_score[i][0]+=0.5
                
    # 4.score based on rating value and rating count
    #we add a score according to the product of the rating value and the rating count
    #we take its logarithm to avoid too different gap 
    #and for not penalizing the recent books which have few rating count
    
    for x in new_score:
        tmp = dataset_query.loc[x[1]]['ratingValue']*dataset_query.loc[x[1]]['ratingCount']
        if tmp < 1: 
            x[0] += 0
        else: 
            x[0] += np.log10(tmp)
            
    # 5.1 score based on the autors
    #the user can insert one or more authors ,only their names or surnames or either,
    #if they are between the books returned by the previous exercise we add a score 
    autor_input=input("4. Inserisci l'autore (o gli autori),il suo nome,il suo cognome o entrambi,\
separati dalla virgola,altrimenti premi invio (e.g. suzanne collins,terry brook)\n").title()
        
    if autor_input!='':
        autor_input=re.findall(r"[\w']+", autor_input)
        
        n_autors=len(autor_input)
        for i in range(n):
            count=0
            for a in autor_input:
                if a in autors.iloc[i]:
                    count+=1
            new_score[i][0]+=count/n_autors
            
    # 5.2 score based in the popularity of each autors
    all_autors=find_authors(df)
    query_autors=find_authors(dataset_query)
    
    
    #create a vocabualry where key:name_autor value:number of book he wrote
    diz_popularity=find_authors_popularity(query_autors,all_autors)
    
    #value that we use to normalize the result
    norm=sum(diz_popularity.values())
    
    for i in range(n):
        aut=autors.iloc[i]
        aut=aut.strip('[]').split(', ')
        s=0
        for a in aut:
            a=a.strip("'")
            s+=diz_popularity[a]
        new_score[i][0]+=(s/norm)
    
        
                                            
    #create a heap data structure            
    H=[]
    for i in range(len(new_score)):
        heapq.heappush(H,(new_score[i][0],new_score[i][1]))
    k=5
    top_k = heapq.nlargest (k, H)
    
    score_similarity_top_k=[]
    document_ID_top_k=[]
    for i in top_k:
        score_similarity_top_k.append(i[0])
        document_ID_top_k.append(i[1])
    
    df_top_k=dataset_query.loc[document_ID_top_k][['bookTitle', 'Plot', 'Url']]
    df_top_k['new_similarity_score']=score_similarity_top_k
    
    return df_top_k
    

In [14]:
top_k_books=new_score_func(results)

1. Qual è il titolo del tuo libro? Inserisci le parole chiave del titolo che stai cercando,altrimenti premi invio (e.g. Harry)
fire
2. Quante pagine ha il libro che cerchi?Inserisci un range di pagine,altrimenti premi invio (e.g. 200-400)
200-400
3. Quando è stato pubblicato il libro?Inserisci un range di anni,altrimenti premi invio(e.g. 1990-2010)
1990-2010
4. Inserisci l'autore (o gli autori),il suo nome,il suo cognome o entrambi,separati dalla virgola,altrimenti premi invio (e.g. suzanne collins,terry brook)
terry brook,suzanne


In [15]:
top_k_books

,bookTitle,Plot,Url,new_similarity_score
23820,The Fire Within,"A magical adventure about the power of dragons...When David moves in with Elizabeth Pennykettle and her eleven-year-old daughter, Lucy, he discovers a collection of clay dragons that come to life. David's own special dragon inspires him to write a story, which reveals the secrets behind a mystery. In order to solve the mystery and save his dragon, David must master the magic of the fire within - not only with his hands but also with his heart.",https://www.goodreads.com/book/show/312043.The_Fire_Within,6.537765
4280,Cold Fire,"There is an alternate cover edition here.Daja and Frostpine expect to have a peaceful winter's visit with old friends in Kugisko, a port in the vast empire of Namon. But there is no peace when mysterious fires begin to blaze across the vulnerable city. Daja assists Bennat Ladradun, a local firefighter with a tragic past, to fight the flames. The two become fast friends-until they realize the fires have been deliberately set, and their relationship is deeply tested. Daja's magic helps her track down the firestarter, but no magic can protect her or Ben from the effects of madness and betrayal.",https://www.goodreads.com/book/show/153792.Cold_Fire,6.463293
1088,The Fire,"Whit and Wisty Allgood have sacrificed everything to lead the resistance against the merciless totalitarian regime that governs their world. Its supreme leader, The One Who Is The One, has banned everything they hold dear: books, music, art, and imagination. But the growing strength of the siblings' magic hasn't been enough to stop the One's evil rampage, and now he's executed the only family they had left.Wisty knows that the time has finally come for her to face The One. But her fight and her fire only channel more power to this already invincible being. How can she and Whit possibly prepare for their imminent showdown with the ruthless villain that devastated their world-before he can truly become all-powerful?In this stunning climax of the epic Witch &amp; Wizard series, the stakes have never been higher-and the consequences will change everything.",https://www.goodreads.com/book/show/10778410-the-fire,6.423133
15716,The Way of Kings,"I long for the days before the Last Desolation. Before the Heralds abandoned us and the Knights Radiant turned against us. When there was still magic in Roshar and honor in the hearts of men.In the end, not war but victory proved the greater test. Did our foes see that the harder they fought, the fiercer our resistance? Fire and hammer forge a sword; time and neglect rust it away. So we won the world, yet lost it.Now there are four whom we watch: the surgeon, forced to forsake healing and fight in the most brutal war of our time; the assassin, who weeps as he kills; the liar, who wears her scholar's mantle over a thief's heart; and the prince, whose eyes open to the ancient past as his thirst for battle wanes. One of them may redeem us. One of them will destroy us.",https://www.goodreads.com/book/show/7235533-the-way-of-kings,6.188298
10312,Brisingr,"Oaths sworn... loyalties tested... forces collide.It's been only months since Eragon first uttered ""brisingr"", an ancient language term for fire. Since then, he's not only learned to create magic with words — he's been challenged to his very core. Following the colossal battle against the Empires warriors on the Burning Plains, Eragon and his dragon, Saphira, have narrowly escaped with their lives. Still, there is more adventure at hand for the Rider and his dragon, as Eragon finds himself bound by a tangle of promises he may not be able to keep.First is Eragon's oath to his cousin, Roran: to help rescue Roran's beloved from King Galbatorix's clutches. But Eragon owes his loyalty to others, too. The Varden are in desperate need of his talents and strength — as are the elves and dwarves. When unrest claims the rebels and danger strikes from every corner, Eragon must make choices — choices that wi